### 42186 - Model Based Machine Learning

Hello, welcome to our course on Model-Based Machine Learning. We hope you will enjoy and learn a lot of new powerful techniques for your future.



# Week1. Probability and statistics review

For a start, we need to review some concepts of probability and statistics. 

As usual in Python, the first thing to do is to import the necessary packages. Let's start with some usual ones for now...

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
%matplotlib inline

## Part 1 - Random variables, independence, conditionality, Bayes theorem

We are going to work on a 2D world for now, and a uniform distribution.

So, let's create N 2D points:

In [3]:
N=500
pts=np.random.uniform(size=(N,2)) 

Take a look at these points. You'll notice they all fall inside a [0,1]x[0,1] square...

In [4]:
pts[:10]

array([[0.6306449 , 0.73302118],
       [0.34141592, 0.55589052],
       [0.59074327, 0.72758844],
       [0.80860934, 0.90572937],
       [0.72855892, 0.69717204],
       [0.61020307, 0.32148889],
       [0.10875998, 0.96692783],
       [0.19032383, 0.73984347],
       [0.28759457, 0.0392651 ],
       [0.1742224 , 0.49052166]])

Can you count how many such points fall inside a circle centered at (0,0) and radius 1?

In [5]:
def in_circ(x,y):
     return x**2+y**2<1   

In [6]:
inc=sum([in_circ(x,y) for x, y in pts])
print(inc)

420

Consider a={a given point (x,y) falls inside the circle centered at (0,0) with radius 1}

Calculate the probability of a, p(a)

Visualizations are often quite useful. Can you visualize both the points as well as the circle line?

Just to link with your trigonometrics knowledge: what is the area of this circle (radius=1)? And what is the area of a quarter of that circle (the part that you just drew)?

What is the relationship with that value and the probability that you just calculated (you can vary the number of points, N, to check your theory)?

Now, let's add a new event b={a point (x,y) falls on the right hand side of the [0,1]x[0,1] square}

Calculate p(b) 

So, now you have p(a) and p(b). Do you want to try calculating p(a,b)?

Are they independent? Is there any reason to believe that they aren't?

To check this, try two approaches:

-- p(a,b)=p(a)*p(b)

-- Just count the points that fulfill both a and b constraints (and divide by the total number of points... ;-) ).


They are different! Why?... If you are in doubt, take a look at the picture...

Maybe they are not independent, after all... if so, their joint probability is instead

p(a,b)=p(a|b)*p(b)


Please calculate p(a|b), in order to get the right value...

Now you calculate p(a,b)=p(a|b)*p(b)

Compare with the values above. Does it make more sense now? 

Another well known property is the Bayes theorem:

$p(b|a)=\frac{p(a|b)p(b)}{p(a)}$

Calculate the p(b|a) using the Bayes theorem, and calculate it directly from data. Are they converging to the same value?


Look at the following table:

In [7]:
f=pd.DataFrame([[0.068966, 0.137931, 0.068966], [0.344828, 0.241379, 0.137931]], columns=['T=Hot', 'T=Mild', 'T=Cold'], index=['W=Sunny', "W=Cloudy"])
print(f)

             T=Hot    T=Mild    T=Cold
W=Sunny   0.068966  0.137931  0.068966
W=Cloudy  0.344828  0.241379  0.137931


Marginalization allows us to compute distributions over a selection of variables that we want. For example, for the table above, that represents p(W, T) in a non-normalized way, we could "marginalize out weather", i.e. calculate p(T), by doing 

$p(T)=\sum_W p(W, T)$

Marginalize out weather (Hint: the pandas package has a nice "sum()" function associated with the the DataFrame object f that you can use...)

In [8]:
f.sum(axis=0)

T=Hot     0.413794
T=Mild    0.379310
T=Cold    0.206897
dtype: float64

Marginalize out temperature

--------

## Part 2 - Independence

Let's get further into the concept of **independence**

To begin with, let's create a small dataset of size N. We will start with two random variables, z and t, both uniformly distributed. 

Each random variable will be an integer value in {0, 1, 2} (you can change this later, of course). 

In [9]:
N=1000
D=3

z=np.random.randint(D, size=N)
t=np.random.randint(D, size=N)

Let's calculate the probability of each different value of z. Take a look at this code. It may be useful later

In [10]:
z_items, z_counts=np.unique(z, return_counts=True, axis=0) #returns a list of different items in Z; it also counts how 
                                                           #many times each one appears

#calculates the p(Z=z), for each possible value of z
for item, ct in zip(z_items, z_counts):     
    print("p(z=%d)=%f"%(item, ct/N))

p(z=0)=0.352000
p(z=1)=0.337000
p(z=2)=0.311000


Actually, we're going to reuse the above code quite a lot. Let's make a new function, then... 

In [11]:
#Function p. It receives a list of values, K, and simply prints the probability of each of its values
#nam and given - Two strings just for printing. Just run this once and you'll see... ;-)
def p(nam, K, given=""):    
    k_items, k_counts=np.unique(K, return_counts=True, axis=0)
    for item, ct in zip(k_items, k_counts):
        print("p(%s=%s%s)=%f"%(nam, item, given, ct/sum(k_counts)))

Let's try it with our variables Z and T

In [12]:
p('z', z)
p('t', t)

p(z=0)=0.352000
p(z=1)=0.337000
p(z=2)=0.311000
p(t=0)=0.329000
p(t=1)=0.332000
p(t=2)=0.339000


ok. So, we have the probability of each value, according to this dataset. 

Assuming that both z and t are uniformly distributed, the calculation for the probability of each should be trivial, right? What is it?

Does it match the above? If you're in doubt, you can increase the value of N, the dataset size. As N grows, it should approximate your calculation... 

Let's now think of the joint distribution of z and t. In other words, how these two variables seem to co-vary, together. The first thing to do is to "align" them, i.e., re-create a new random variable (z_t), that is a pair of observations, z and t. 

In [13]:
z_t=np.stack((z,t), axis=-1)  #Check the result of this operation, to understand what it does...

So, let's use the function above, to calculate the join distribution

In [14]:
p("z,t", z_t)

p(z,t=[0 0])=0.123000
p(z,t=[0 1])=0.119000
p(z,t=[0 2])=0.110000
p(z,t=[1 0])=0.114000
p(z,t=[1 1])=0.106000
p(z,t=[1 2])=0.117000
p(z,t=[2 0])=0.092000
p(z,t=[2 1])=0.107000
p(z,t=[2 2])=0.112000


Are Z and T independent? If yes, the formula for their join distribution is trivial. Please calculate it (and check if it verifies).

Let's create two more variables, both based on Z. 

In [15]:
betay=-2    
betax=1
y=betay*z
x=betax*z

Are they independent? What do you think? A good way to see is by plotting... do you want to do it?

What should it look like when variables are independent? And when they are correlated?

Let's check their joint distribution, then... 

In [16]:
x_y=np.stack((x,y), axis=-1)
p("x,y", x_y)

p(x,y=[0 0])=0.352000
p(x,y=[ 1 -2])=0.337000
p(x,y=[ 2 -4])=0.311000


Notice that, for all other value combinations (e.g. x,y=[0,-2]), the probability is zero.

If x and y are independent, then the product of their marginal probabilities should be equal to the values above, right? Do you want to calculate?

Should the values match at all? Let's try instead to calculate the conditional probabilities, of and x and y, given z.

To do it, directly on the data, we need to calculate, for EACH value of z, the distributions for both x and y.

This means that we need to organize the data accordingly. How about using a dictionary? 

In [17]:
from collections import defaultdict

z_dict=defaultdict(list)

for X, Y, Z in zip(x, y, z):
    z_dict[Z].append([X,Y])
    

To make sure you understand the code above, don't forget to check z_dict carefully...

The code below uses the dictionary to calculate the conditional marginal and joint distributions. Check it carefully.

In [19]:
for Z in z_dict:
    x_y=z_dict[Z]
    x=np.array(x_y)[:,0]
    y=np.array(x_y)[:,1]  
    p("x", x, "|z=%d"%Z)
    p("y", y,"|z=%d"%Z)
    p("x,y", x_y, "|z=%d"%Z)

p(x=1|z=1)=1.000000
p(y=-2|z=1)=1.000000
p(x,y=[ 1 -2]|z=1)=1.000000
p(x=0|z=0)=1.000000
p(y=0|z=0)=1.000000
p(x,y=[0 0]|z=0)=1.000000
p(x=2|z=2)=1.000000
p(y=-4|z=2)=1.000000
p(x,y=[ 2 -4]|z=2)=1.000000


So, what do you conclude? Is it true that x is independent of y given z?

The example above was particularly odd, because there is NO randomness involved (i.e. x is **exactly** betax times z). As a consequence, the probabilities are quite extreme. 

Try to redo the whole exercise, by adding a little randomness to x and y (don't forget to keep them as integers). Notice how the conditional independence property varies of "intensity", as you add more or less randomness... 

Finally, let's try a different perspective on conditional independence: why does it actually matter??

Particularly, we want to understand the meaning of the following sentence:

**"If we know z, then knowing about y tells us nothing about x"**



Using your small dataset (x, y, z), you will try to make linear regression prediction models. Our goal is to "know about x", i.e. x will be the tar get (or dependent) variable in our linear regression. 

Compare the following models (e.g. using sklearn's LinearRegression model):

- $x=\beta_1*z$
- $x=\beta_1*y+\beta_2*z$
- $x=\beta_1*y$

Notice the coefficients, what do they mean?

--------

## Part 3 - Expectation of discrete variables

Calculate the expected value of x, using

$E(x)=\sum_x xp(x)$

Compare it with the mean of x (x.mean() )

-------